In [19]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

from collections import OrderedDict
from random import randint
import random
import decimal
from operator import add

In [2]:
max_hidden_layers = 9
max_neurons_per_layer = 100
min_learning_rate = .0001
max_learning_rate = .01
acti_func = ["softmax", "elu", "selu", "softplus", "softsign", 
             "relu", "tanh", "sigmoid", "hard_sigmoid", "linear"]
num_individuals = 4
loss_weight = 10
accuracy_weight = 10
time_weight = 10

In [3]:
def load_data():
    iris_data = load_iris()
    print('Example Data:')
    print(iris_data.data[:5])
    print('Example Labels:')
    print(iris_data.target[:5])
    global iris_data

<ipython-input-3-11c664384486>:7: SyntaxWarning: name 'iris_data' is assigned to before global declaration
  global iris_data


In [4]:
def process_data():
    x = iris_data.data
    y_ = iris_data.target.reshape(-1,1)

    encoder = OneHotEncoder(sparse=False)
    y = encoder.fit_transform(y_)
    train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=.20)
    global train_x, test_x, train_y, test_y

<ipython-input-4-1082ec30e327>:8: SyntaxWarning: name 'train_x' is assigned to before global declaration
  global train_x, test_x, train_y, test_y
<ipython-input-4-1082ec30e327>:8: SyntaxWarning: name 'test_x' is assigned to before global declaration
  global train_x, test_x, train_y, test_y
<ipython-input-4-1082ec30e327>:8: SyntaxWarning: name 'train_y' is assigned to before global declaration
  global train_x, test_x, train_y, test_y
<ipython-input-4-1082ec30e327>:8: SyntaxWarning: name 'test_y' is assigned to before global declaration
  global train_x, test_x, train_y, test_y


In [5]:
load_data()
process_data()

Example Data:
[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]
Example Labels:
[0 0 0 0 0]


In [30]:
def create_network(model_num):
    model_name = "Model"+str(model_num)
    model_desc = {'name':model_name}
    print(model_name)
    model_name = Sequential()
    num_hidden_layers = randint(0, max_hidden_layers)
    i = 0
    model_name.add(Dense(10, input_shape=(4,), activation='relu', name='input'))
    while i < num_hidden_layers:
        name = 'hidden'+str(i)
        num_neurons = randint(1, max_neurons_per_layer)
        activation = acti_func[randint(0, len(acti_func)-1)]
        model_desc.update({'hidden'+str(i) : (num_neurons, activation)})
        #print(activation)
        model_name.add(Dense(num_neurons, activation=activation, name=name))
        i = i+1
    model_name.add(Dense(3, activation='softmax', name='output'))
    global model_name
    global model_desc

<ipython-input-30-4e0afccb8382>:18: SyntaxWarning: name 'model_name' is assigned to before global declaration
  global model_name
<ipython-input-30-4e0afccb8382>:19: SyntaxWarning: name 'model_desc' is assigned to before global declaration
  global model_desc


In [31]:
def lr_and_optimizer(model_name):
    learning_rate = random.uniform(min_learning_rate, max_learning_rate)
    #print(learning_rate)
    model_desc.update({'learning_rate' : learning_rate})
    optimizer = Adam(lr=learning_rate)
    model_desc.update({'optimizer' : 'Adam'})
    model_name.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [32]:
def print_summary(model_name):
    print('Neural Net Model Summary: ')
    print(model_name.summary())

In [33]:
def train_and_test(model_name):
    model_name.fit(train_x, train_y, verbose = 2, batch_size = 5, epochs = 200)
    results = model_name.evaluate(test_x, test_y)
    global results

<ipython-input-33-915b75c55b69>:4: SyntaxWarning: name 'results' is assigned to before global declaration
  global results


In [34]:
def individual(model_num):
    create_network(model_num)
    #print_summary(model_name)
    lr_and_optimizer(model_name)
    train_and_test(model_name)
    loss_fitness =  (1/float(results[0]))*loss_weight
    acc_fitness = float(results[1])*accuracy_weight
    total_fitness = loss_fitness + acc_fitness
    model_desc.update({'loss_fitness':loss_fitness})
    model_desc.update({'accuracy_fitness':acc_fitness})
    model_desc.update({'total_fitness':total_fitness})
    print(model_name)
    #print(model_desc)

In [35]:
def population():
    population_desc = []
    for x in xrange(num_individuals):
        individual(model_num = x)
        population_desc.append(model_desc)
    print(population_desc)
    global population_desc

<ipython-input-35-662326261e74>:7: SyntaxWarning: name 'population_desc' is assigned to before global declaration
  global population_desc


In [36]:
population()

Model0
sigmoid
softmax
linear
softsign
sigmoid
tanh
softplus
Neural Net Model Summary: 
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (Dense)                (None, 10)                50        
_________________________________________________________________
hidden0 (Dense)              (None, 82)                902       
_________________________________________________________________
hidden1 (Dense)              (None, 55)                4565      
_________________________________________________________________
hidden2 (Dense)              (None, 13)                728       
_________________________________________________________________
hidden3 (Dense)              (None, 90)                1260      
_________________________________________________________________
hidden4 (Dense)              (None, 7)                 637       
______________________________________________________

 - 0s - loss: 0.0977 - acc: 0.9417
Epoch 138/200
 - 0s - loss: 0.0716 - acc: 0.9750
Epoch 139/200
 - 0s - loss: 0.1075 - acc: 0.9500
Epoch 140/200
 - 0s - loss: 0.0858 - acc: 0.9667
Epoch 141/200
 - 0s - loss: 0.0751 - acc: 0.9750
Epoch 142/200
 - 0s - loss: 0.0764 - acc: 0.9583
Epoch 143/200
 - 0s - loss: 0.1053 - acc: 0.9583
Epoch 144/200
 - 0s - loss: 0.1068 - acc: 0.9667
Epoch 145/200
 - 0s - loss: 0.0566 - acc: 0.9667
Epoch 146/200
 - 0s - loss: 0.1636 - acc: 0.9333
Epoch 147/200
 - 0s - loss: 0.1155 - acc: 0.9667
Epoch 148/200
 - 0s - loss: 0.0833 - acc: 0.9667
Epoch 149/200
 - 0s - loss: 0.0928 - acc: 0.9583
Epoch 150/200
 - 0s - loss: 0.1113 - acc: 0.9583
Epoch 151/200
 - 0s - loss: 0.0804 - acc: 0.9750
Epoch 152/200
 - 0s - loss: 0.0687 - acc: 0.9667
Epoch 153/200
 - 0s - loss: 0.0620 - acc: 0.9833
Epoch 154/200
 - 0s - loss: 0.0634 - acc: 0.9750
Epoch 155/200
 - 0s - loss: 0.0703 - acc: 0.9750
Epoch 156/200
 - 0s - loss: 0.0888 - acc: 0.9583
Epoch 157/200
 - 0s - loss: 0.0843

Epoch 81/200
 - 0s - loss: 0.0751 - acc: 0.9667
Epoch 82/200
 - 0s - loss: 0.0553 - acc: 0.9917
Epoch 83/200
 - 0s - loss: 0.0936 - acc: 0.9667
Epoch 84/200
 - 0s - loss: 0.2737 - acc: 0.9083
Epoch 85/200
 - 0s - loss: 0.1396 - acc: 0.9500
Epoch 86/200
 - 0s - loss: 0.0880 - acc: 0.9750
Epoch 87/200
 - 0s - loss: 0.0615 - acc: 0.9833
Epoch 88/200
 - 0s - loss: 0.0546 - acc: 0.9750
Epoch 89/200
 - 0s - loss: 0.0633 - acc: 0.9750
Epoch 90/200
 - 0s - loss: 0.0954 - acc: 0.9500
Epoch 91/200
 - 0s - loss: 0.0750 - acc: 0.9833
Epoch 92/200
 - 0s - loss: 0.0614 - acc: 0.9750
Epoch 93/200
 - 0s - loss: 0.0674 - acc: 0.9667
Epoch 94/200
 - 0s - loss: 0.0712 - acc: 0.9833
Epoch 95/200
 - 0s - loss: 0.0782 - acc: 0.9583
Epoch 96/200
 - 0s - loss: 0.0563 - acc: 0.9833
Epoch 97/200
 - 0s - loss: 0.0646 - acc: 0.9833
Epoch 98/200
 - 0s - loss: 0.0579 - acc: 0.9750
Epoch 99/200
 - 0s - loss: 0.1032 - acc: 0.9667
Epoch 100/200
 - 0s - loss: 0.1040 - acc: 0.9583
Epoch 101/200
 - 0s - loss: 0.0914 - ac

Epoch 12/200
 - 0s - loss: 0.5408 - acc: 0.6500
Epoch 13/200
 - 0s - loss: 0.5242 - acc: 0.6667
Epoch 14/200
 - 0s - loss: 0.5141 - acc: 0.6250
Epoch 15/200
 - 0s - loss: 0.5051 - acc: 0.6500
Epoch 16/200
 - 0s - loss: 0.5025 - acc: 0.6500
Epoch 17/200
 - 0s - loss: 0.4984 - acc: 0.6250
Epoch 18/200
 - 0s - loss: 0.5107 - acc: 0.6667
Epoch 19/200
 - 0s - loss: 0.4915 - acc: 0.6167
Epoch 20/200
 - 0s - loss: 0.4947 - acc: 0.6250
Epoch 21/200
 - 0s - loss: 0.4901 - acc: 0.6250
Epoch 22/200
 - 0s - loss: 0.4855 - acc: 0.6250
Epoch 23/200
 - 0s - loss: 0.4841 - acc: 0.6667
Epoch 24/200
 - 0s - loss: 0.4843 - acc: 0.6583
Epoch 25/200
 - 0s - loss: 0.4851 - acc: 0.6417
Epoch 26/200
 - 0s - loss: 0.4859 - acc: 0.6667
Epoch 27/200
 - 0s - loss: 0.4818 - acc: 0.6500
Epoch 28/200
 - 0s - loss: 0.4850 - acc: 0.6667
Epoch 29/200
 - 0s - loss: 0.4960 - acc: 0.6083
Epoch 30/200
 - 0s - loss: 0.4883 - acc: 0.6083
Epoch 31/200
 - 0s - loss: 0.4837 - acc: 0.6083
Epoch 32/200
 - 0s - loss: 0.4790 - acc:

Epoch 181/200
 - 0s - loss: 0.2392 - acc: 0.9250
Epoch 182/200
 - 0s - loss: 0.0981 - acc: 0.9750
Epoch 183/200
 - 0s - loss: 0.0893 - acc: 0.9833
Epoch 184/200
 - 0s - loss: 0.0809 - acc: 0.9833
Epoch 185/200
 - 0s - loss: 0.0828 - acc: 0.9750
Epoch 186/200
 - 0s - loss: 0.0815 - acc: 0.9750
Epoch 187/200
 - 0s - loss: 0.1189 - acc: 0.9667
Epoch 188/200
 - 0s - loss: 0.1173 - acc: 0.9500
Epoch 189/200
 - 0s - loss: 0.1550 - acc: 0.9500
Epoch 190/200
 - 0s - loss: 0.1048 - acc: 0.9750
Epoch 191/200
 - 0s - loss: 0.1432 - acc: 0.9500
Epoch 192/200
 - 0s - loss: 0.1397 - acc: 0.9500
Epoch 193/200
 - 0s - loss: 0.1194 - acc: 0.9667
Epoch 194/200
 - 0s - loss: 0.1124 - acc: 0.9667
Epoch 195/200
 - 0s - loss: 0.1006 - acc: 0.9667
Epoch 196/200
 - 0s - loss: 0.0686 - acc: 0.9750
Epoch 197/200
 - 0s - loss: 0.0584 - acc: 0.9833
Epoch 198/200
 - 0s - loss: 0.0586 - acc: 0.9917
Epoch 199/200
 - 0s - loss: 0.2448 - acc: 0.9333
Epoch 200/200
 - 0s - loss: 0.1791 - acc: 0.9583
30/30 [=============

Epoch 122/200
 - 0s - loss: 0.0641 - acc: 0.9667
Epoch 123/200
 - 0s - loss: 0.0781 - acc: 0.9750
Epoch 124/200
 - 0s - loss: 0.0737 - acc: 0.9667
Epoch 125/200
 - 0s - loss: 0.0652 - acc: 0.9750
Epoch 126/200
 - 0s - loss: 0.0819 - acc: 0.9583
Epoch 127/200
 - 0s - loss: 0.0653 - acc: 0.9750
Epoch 128/200
 - 0s - loss: 0.0709 - acc: 0.9750
Epoch 129/200
 - 0s - loss: 0.0687 - acc: 0.9667
Epoch 130/200
 - 0s - loss: 0.0597 - acc: 0.9833
Epoch 131/200
 - 0s - loss: 0.0649 - acc: 0.9667
Epoch 132/200
 - 0s - loss: 0.0663 - acc: 0.9750
Epoch 133/200
 - 0s - loss: 0.0857 - acc: 0.9667
Epoch 134/200
 - 0s - loss: 0.0819 - acc: 0.9667
Epoch 135/200
 - 0s - loss: 0.0644 - acc: 0.9667
Epoch 136/200
 - 0s - loss: 0.0559 - acc: 0.9917
Epoch 137/200
 - 0s - loss: 0.0653 - acc: 0.9750
Epoch 138/200
 - 0s - loss: 0.0517 - acc: 0.9833
Epoch 139/200
 - 0s - loss: 0.0669 - acc: 0.9750
Epoch 140/200
 - 0s - loss: 0.0450 - acc: 0.9833
Epoch 141/200
 - 0s - loss: 0.0553 - acc: 0.9750
Epoch 142/200
 - 0s 

In [44]:
def grade():    
    fitness_sum = 0
    'Find average fitness for a population.'
    for x in population_desc:
        fitness_sum = x.get("total_fitness") + fitness_sum
    avg_fitness = fitness_sum/len(population_desc)
    print(avg_fitness)

In [45]:
grade()

329.173970474
